# Endpoint Selector — Jupyter Option
Send a test chat request to different OpenAI-compatible endpoints.


In [ ]:
from openai import OpenAI
import os
import ipywidgets as widgets
from IPython.display import display, Markdown


In [ ]:
endpoints = {
    'Ollama (local)': 'http://localhost:11434/v1',
    'LM Studio (local)': 'http://localhost:1234/v1',
    'Remote (edit IP)': 'http://192.168.1.50:11434/v1',
    'OpenAI (cloud)': 'https://api.openai.com/v1',
}

endpoint_dd = widgets.Dropdown(options=list(endpoints.keys()), description='Endpoint:')
base_url_txt = widgets.Text(value=endpoints[endpoint_dd.value], description='Base URL:')
model_txt = widgets.Text(value='llama3.2:1b', description='Model:')
msg_txt = widgets.Textarea(
    value='Explain what an API is in 2 sentences for a business student.',
    description='Prompt:',
    layout=widgets.Layout(width='95%', height='90px')
)
key_txt = widgets.Password(value=os.getenv('OPENAI_API_KEY',''), description='API key:')
send_btn = widgets.Button(description='Send')
out = widgets.Output()

def on_endpoint_change(change):
    base_url_txt.value = endpoints[endpoint_dd.value]

endpoint_dd.observe(on_endpoint_change, names='value')

def do_send(_):
    out.clear_output()
    base_url = base_url_txt.value.strip()
    api_key = key_txt.value.strip() or 'ollama'
    client = OpenAI(base_url=base_url, api_key=api_key)
    try:
        resp = client.chat.completions.create(
            model=model_txt.value.strip(),
            messages=[{'role':'user','content': msg_txt.value}],
            temperature=0.2,
        )
        text = resp.choices[0].message.content
        with out:
            display(Markdown('### Response'))
            display(Markdown(text))
    except Exception as e:
        with out:
            display(Markdown('### Error'))
            display(Markdown(f'`{type(e).__name__}`: {e}'))

send_btn.on_click(do_send)
display(endpoint_dd, base_url_txt, model_txt, msg_txt, key_txt, send_btn, out)
